# Adding Other Features

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
daywisedf = pd.read_csv("Week_df.csv", index_col=0)

In [ ]:
def adapterToPool(dataframe, function, supply=[]):

    if len(supply) == 0:
        supply = getSupply(dataframe)

    hold_values = []

    try:
        pool = Pool(num_pool_to_use)
        hold_values = pool.map(function, supply)
    finally:
        pool.close()
        pool.join()

    return pd.DataFrame(hold_values)


def getSupply(dataframe):
    supply = []
    for x in range(len(dataframe)):
        supply.append((x, dataframe.iloc[x]))

    return supply

In [2]:
def oneHotEncoder(df, col):
    lb_style = LabelBinarizer()
    lb_results = lb_style.fit_transform(df[col])
    lb_new = pd.DataFrame(lb_results, columns=lb_style.classes_)
    col_names = lb_new.columns
    new_col_names = []
    for i in col_names:
        new_col_names.append((col+'_'+str(i)))
    lb_new.columns = new_col_names
    df = pd.concat([df, lb_new], axis=1)
    return df


def addCapFreq(week_df):
    show_wise_df = pd.read_csv("Daywise_model.csv")
    show_wise_df['dateofshow'] = pd.to_datetime(
        show_wise_df['DateTime']).dt.date
    lb_style = LabelBinarizer()
    lb_results = lb_style.fit_transform(show_wise_df["Capacity"])
    lb_Capacity = pd.DataFrame(lb_results, columns=lb_style.classes_)
    print(lb_Capacity.columns)
    lb_Capacity.columns = ['Cap_110', 'Cap_120', 'Cap_131', 'Cap_310']
    show_wise_df = pd.concat([show_wise_df, lb_Capacity], axis=1)
    capfreq = show_wise_df.groupby(['dateofshow', 'Movie'], as_index=False).aggregate(
        {'Cap_110': 'sum', 'Cap_120': 'sum', 'Cap_131': 'sum', 'Cap_310': 'sum'})  # Same as above
    week_df = pd.concat(
        [week_df, capfreq[['Cap_110', 'Cap_120', 'Cap_131', 'Cap_310']]], axis=1)
    return week_df


def addScreenFreq(week_df):
    show_wise_df = pd.read_csv("Daywise_model.csv")
    show_wise_df['dateofshow'] = pd.to_datetime(
        show_wise_df['DateTime']).dt.date
    lb_style = LabelBinarizer()
    lb_results = lb_style.fit_transform(show_wise_df["Screen"])
    lb_Screens = pd.DataFrame(lb_results, columns=lb_style.classes_)
    show_wise_df = pd.concat([show_wise_df, lb_Screens], axis=1)
    screenfreq = show_wise_df.groupby(['dateofshow', 'Movie'], as_index=False).aggregate(
        {'BLUSH': 'sum', 'CARVE': 'sum', 'FRAME': 'sum', 'KITES': 'sum', 'PLUSH': 'sum', 'SPOT': 'sum', 'STREAK': 'sum', 'WEAVE': 'sum'})  # Same as above
    week_df = pd.concat([week_df, screenfreq[[
        'BLUSH', 'CARVE', 'FRAME', 'KITES', 'PLUSH', 'SPOT', 'STREAK', 'WEAVE']]], axis=1)
    return week_df


def addTransFreq(week_df):
    transdf = pd.read_csv("Daywise_model", index_col=0)
    escapeScreens = ['STREAK', 'PLUSH', 'FRAME',
                     'SPOT', 'KITES', 'CARVE', 'WEAVE', 'BLUSH']
    transdf = transdf[transdf.screen_strname.isin(escapeScreens)]
    transdf.dropna(inplace=True)
    import datetime as dt
    transdf['transt_dtmdatetime'] = pd.to_datetime(
        transdf['transt_dtmdatetime'])
    transdf['session_dtmshowing'] = pd.to_datetime(
        transdf['session_dtmshowing'])
    transdf = transdf.groupby(['film_strcode', 'session_dtmshowing', 'transt_dtmdatetime',
                               'screen_strname'], as_index=False).aggregate({'transt_intnoofseats': 'sum'})  # Same as above
    transdf = transdf.groupby(['film_strcode', 'session_dtmshowing', 'transt_dtmdatetime',
                               'screen_strname'], as_index=False).aggregate({'transt_intnoofseats': 'sum'})  # Same as above
    transdf = transdf[(transdf["transt_intnoofseats"] >= 1)]
    transdf = transdf[(transdf["transt_intnoofseats"] <= 20)]
    transdf = oneHotEncoder(transdf, 'transt_intnoofseats')
    transdf = transdf.dropna()
    transdf['dateofshow'] = pd.to_datetime(
        transdf['session_dtmshowing'].dt.date)
    transdf.columns = ['Movie', 'session_dtmshowing', 'transt_dtmdatetime', 'screen_strname', 'transt_intnoofseats', 'transt_intnoofseats_1', 'transt_intnoofseats_2', 'transt_intnoofseats_3', 'transt_intnoofseats_4', 'transt_intnoofseats_5', 'transt_intnoofseats_6', 'transt_intnoofseats_7', 'transt_intnoofseats_8',
                       'transt_intnoofseats_9', 'transt_intnoofseats_10', 'transt_intnoofseats_11', 'transt_intnoofseats_12', 'transt_intnoofseats_13', 'transt_intnoofseats_14', 'transt_intnoofseats_15', 'transt_intnoofseats_16', 'transt_intnoofseats_17', 'transt_intnoofseats_18', 'transt_intnoofseats_19', 'transt_intnoofseats_20', 'dateofshow']
    types_list = ['transt_intnoofseats_1', 'transt_intnoofseats_2', 'transt_intnoofseats_3', 'transt_intnoofseats_4', 'transt_intnoofseats_5', 'transt_intnoofseats_6', 'transt_intnoofseats_7', 'transt_intnoofseats_8', 'transt_intnoofseats_9', 'transt_intnoofseats_10',
                  'transt_intnoofseats_11', 'transt_intnoofseats_12', 'transt_intnoofseats_13', 'transt_intnoofseats_14', 'transt_intnoofseats_15', 'transt_intnoofseats_16', 'transt_intnoofseats_17', 'transt_intnoofseats_18', 'transt_intnoofseats_19', 'transt_intnoofseats_20']
    types_agg = dict()
    for i in types_list:
        types_agg[i] = 'sum'
    transdf = transdf.groupby(['dateofshow', 'Movie'], as_index=False).aggregate(
        types_agg)  # Same as above
    transdf.to_csv('DaywiseTransFreq')
    del transdf
    transdf = pd.read_csv('DaywiseTransFreq', index_col=0)
    week_df = pd.merge(week_df, transdf, left_on=[
        'Movie', 'dateofshow'], right_on=['Movie', 'dateofshow'], how='left')
    week_df = week_df.fillna(0)
    return week_df


def addSeatsSoldPerTrans(week_df):
    showwise = pd.read_csv("Daywise_model.csv", index_col=0)
    showwise = showwise.groupby(["Movie", "Date"], as_index=False).aggregate({
        "SeatsSoldPerTrans": "mean"})
    week_df = pd.merge(week_df, showwise, how='left', left_on=[
        "Movie", "dateofshow"], right_on=["Movie", "Date"])
    return week_df


def addLang(week_df):
    lang = pd.read_csv("Movie_Details.csv", index_col=0)
    lang = lang[['MCode', 'Language']]
    week_df = pd.merge(week_df, lang, left_on='Movie',
                       right_on='MCode', how='left')
    week_df.drop(['MCode'], axis=1, inplace=True)
    week_df = week_df.fillna(-1)
    return week_df


def addRelativeOccupancy(week_wise_df):
    tot_occ_week = week_wise_df.groupby(
        ['WeekOfShow'], as_index=False).aggregate({'OccAtWeek': 'sum'})
    tot_occ_week.rename(columns={'OccAtWeek': 'TotOccAtWeek'}, inplace=True)
    week_wise_df = pd.merge(week_df, tot_occ_week, on=[
                            'WeekOfShow'], how='left', copy=False)
    week_wise_df['RelativeOccAtWeek'] = week_wise_df['OccAtWeek'] / \
        week_wise_df['TotOccAtWeek']
    return week_wise_df


def getSSPTHistory(supply, num_history_points=7):

    index, row = supply
    min_date = row['dateofshow'] - timedelta(days=1)
    history = norm_df[norm_df['Movie'] == row['Movie']][(
        norm_df['dateofshow'] <= min_date)].sort_values(by='dateofshow', ascending=False)

    history_pers = history['SeatsSoldPerTrans'].values[:num_history_points]
    for i in range(num_history_points):
        try:
            row[f"SeatsSoldPerTrans{i}"] = history_pers[i]
        except IndexError:
            row[f"SeatsSoldPerTrans{i}"] = -1.0

    try:
        row['SeatsSoldPerTrans_History_Mean'] = statistics.mean(history_pers)
        row['SeatsSoldPerTrans_History_Max'] = max(history_pers)
        row['SeatsSoldPerTrans_History_Min'] = min(history_pers)
    except:
        row['SeatsSoldPerTrans_History_Mean'] = -1
        row['SeatsSoldPerTrans_History_Max'] = -1
        row['SeatsSoldPerTrans_History_Min'] = -1
    return row


def getRelativeHistory(supply, num_history_points=7):

    index, row = supply
    min_date = row['WeekOfShow']
    history = week_df[week_df['Movie'] == row['Movie']][(
        week_df['WeekOfShow'] <= min_date)].sort_values(by='WeekOfShow', ascending=False)

    history_pers = history['RelativeOccAtWeek'].values[:num_history_points]
    for i in range(num_history_points):
        try:
            row[f"RelativeOccAtWeek{i}"] = history_pers[i]
        except IndexError:
            row[f"RelativeOccAtWeek{i}"] = -1.0

    try:
        row['RelativeOccAtWeek_History_Mean'] = statistics.mean(history_pers)
        row['RelativeOccAtWeek_History_Max'] = max(history_pers)
        row['RelativeOccAtWeek_History_Min'] = min(history_pers)
    except:
        row['RelativeOccAtWeek_History_Mean'] = -1
        row['RelativeOccAtWeek_History_Max'] = -1
        row['RelativeOccAtWeek_History_Min'] = -1
    return row


def addHistoryFeatures(week_wise_df):
    df = pd.read_csv('Daywise_model.csv', index_col=0)
    norm_df = df
    supply = getSupply(df[['Movie', 'dateofshow', 'SeatsSoldPerTrans']])
    df['dateofshow'] = pd.to_datetime(df['dateofshow']).dt.date
    df = adapterToPool(df, getSSPTHistory)
    print("Added Show History Features")
    df['dateofshow'] = pd.to_datetime(df['dateofshow']).dt.date
    week_matrix_daywise['date'] = pd.to_datetime(
        week_matrix_daywise['date']).dt.date
    df = pd.merge(df, week_matrix_daywise[[
                  'date', 'week']], how='left', left_on='dateofshow', right_on='date')
    df1 = df.groupby(['Movie', 'week'], as_index=False).aggregate(
        {'dateofshow': 'max'})
    df1 = pd.merge(df[['Movie', 'dateofshow', 'ShowHistory0', 'ShowHistory1', 'ShowHistory2',
                       'ShowHistory3', 'ShowHistory4', 'ShowHistory5', 'ShowHistory6',
                       'WalkinHistory0', 'WalkinHistory1', 'WalkinHistory_Mean',
                       'WalkinHistory_Max', 'WalkinHistory_Min', 'History_Mean',
                       'WalkinHistory2', 'WalkinHistory3', 'WalkinHistory4', 'WalkinHistory5',
                       'WalkinHistory6', 'SeatsSoldPerTrans', 'SeatsSoldPerTrans0',
                       'SeatsSoldPerTrans1', 'SeatsSoldPerTrans2', 'SeatsSoldPerTrans3',
                       'SeatsSoldPerTrans4', 'SeatsSoldPerTrans5', 'SeatsSoldPerTrans6',
                       'SeatsSoldPerTrans_History_Mean', 'SeatsSoldPerTrans_History_Max',
                       'SeatsSoldPerTrans_History_Min']], df1, how='right', on=['Movie', 'dateofshow'])
    week_wise_df = pd.merge(week_wise_df, df1.drop('dateofshow', axis=1), how='left', left_on=[
                            'Movie', 'WeekOfShow'], right_on=['Movie', 'week'])
    print("Added Walkin History Features")
    return week_wise_df

In [10]:
week_df = addTransFreq(week_df)
week_df = addCapFreq(week_df)
week_df = addScreenFreq(week_df)
week_df = addLang(week_df)
week_df = addSeatsSoldPerTrans(week_df)
week_df = addRelativeOccupancy(week_df)

week_df = addRelativeOccupancy(week_df)
week_df = addHistoryFeatures(week_df)
week_df = adapterToPool(week_df, getRelativeHistory)

/opt/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (3,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
week_df.to_csv('Week_df_features.csv')